In [17]:
import pandas as pd
# from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import FirefoxOptions 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database

In [231]:
url = 'https://racetozerodataexplorer.org/proceed/'

scope1_lst = []
scope2_local_lst = []
scope2_market_lst = []
name_lst = []

master_lst = [scope1_lst, scope2_local_lst, scope2_market_lst, name_lst]

options = FirefoxOptions()
options.add_argument("--headless") # comment out to see how it works

with webdriver.Firefox(options=options) as driver:
    driver.get(url)
    headers = driver.find_elements(By.XPATH, "//*[@class= 'styles_table__jWPKo']/thead/tr/th")
    headers = [header.text for header in headers]
    table = driver.find_element(By.TAG_NAME, "table").find_element(By.TAG_NAME, "tbody").find_elements(By.TAG_NAME, 'tr')
    
    for row in table:
        name = row.find_element(By.TAG_NAME, 'th')
        # print(name.text)
        master_lst[3].append(name.text)
        for i, cell in enumerate(row.find_elements(By.TAG_NAME, 'td')):
            if i in [0,1,2]: # only interested in first 3 columns
                try:
                    cell.find_element(By.CLASS_NAME, 'styles_button__IS06I').click()
                    WebDriverWait(driver, 10).until( # Wait for pop-up to load
                        EC.presence_of_element_located((By.CLASS_NAME, 'styles_axisValue__ipOq1')))
                    values = driver.find_elements(By.CLASS_NAME, 'styles_axisValue__ipOq1')
                    master_lst[i].append([value.text.replace(',','') for value in values])
                    driver.find_element(By.CLASS_NAME, 'styles_close__cpVjT').click()
                except:
                    master_lst[i].append(['', '', '', ''])

In [213]:
industries = [
    'Basic Materials',
    'Communication Services',
    'Consumer Cyclicals',
    'Consumer Discretionary',
    'Consumer Non-Cyclicals',
    'Consumer Staples',
    'Energy',
    'Health Care',
    'Industrials',
    'Information Technology',
    'Materials',
    'Real Estate',
    'Technology',
    'Utilities',
]

sectors = []

with webdriver.Firefox(options=options) as driver:
    driver.get(url)
    for industry in industries:
        filter = driver.find_element(By.ID, 'react-select-q-input')
        filter.click()
        filter.send_keys(industry)
        time.sleep(0.2)
        if industry in ['Materials', 'Technology']:
            filter.send_keys(Keys.DOWN)
        filter.send_keys(Keys.ENTER)
        time.sleep(0.2)
        table = driver.find_element(By.TAG_NAME, "table").find_element(By.TAG_NAME, 'tbody')
        names = table.find_elements(By.TAG_NAME, 'th')
        for name in names:
            sectors.append([industry, name.text])
        driver.find_element(By.CLASS_NAME, 'css-g010ao').click()
    

In [149]:
headers = [header.strip().replace('\n', ' ').replace(
    '4 years trend (ending in reporting year)', '') for header in headers]
years = ['2018', '2019', '2020', '2021']
    
# Need to merge all these 
df_scope1 = pd.DataFrame(master_lst[0], index=master_lst[3], columns=years).reset_index(drop=False)
df_scope2_loc = pd.DataFrame(master_lst[1], index=master_lst[3], columns=years).reset_index(drop=False)
df_scope2_mkt = pd.DataFrame(master_lst[2], index=master_lst[3], columns=years).reset_index(drop=False)

df_sectors = pd.DataFrame(master_lst[4], columns=['Industry'], index=master_lst[3])

NameError: name 'headers' is not defined

In [97]:
engine = create_engine("sqlite:///GDS.db")
if not database_exists(engine.url):
    create_database(engine.url)

In [4]:
# Add dataframes to database to avoid having to rerun webscraping each time
for df, header in [(df_scope1, headers[1]), (df_scope2_loc, headers[2]), (df_scope2_mkt, headers[3])]:
    df.rename(columns={'index': 'Company Name'}, inplace=True)
    df.to_sql(header.replace(' ', '_'), engine, if_exists='replace', index=False)

df_sectors.to_sql('Sectors', engine, if_exists="replace", index=False) 

In [171]:
df_scope1 = pd.read_sql_table('Scope_1_emissions_(tCO₂e)', engine)
df_scope2_loc = pd.read_sql_table('Scope_2_emissions_(tCO₂e)_Location', engine)
df_scope2_mkt = pd.read_sql_table('Scope_2_emissions_(tCO₂e)_Market', engine)

df_sectors = pd.read_sql_table('Sectors', engine)

In [172]:
# Drop Interpublic Group of Companies, Inc. which is duplicated in data source
df_scope1 = df_scope1.loc[~df_scope1['Company Name'].duplicated()] 
df_scope2_loc = df_scope2_loc.loc[~df_scope2_loc['Company Name'].duplicated()] 
df_scope2_mkt = df_scope2_mkt.loc[~df_scope2_mkt['Company Name'].duplicated()] 

In [173]:
# Filter out companies with no data
df_scope1 = df_scope1.loc[~df_scope1.iloc[:,1:].duplicated()]
df_scope2_loc = df_scope2_loc.loc[~df_scope2_loc.iloc[:,1:].duplicated()]
df_scope2_mkt = df_scope2_mkt.loc[~df_scope2_mkt.iloc[:,1:].duplicated()]

In [174]:
df_scope1 = df_scope1.melt(id_vars='Company Name', value_name='Scope_1_emissions_(tCO₂e)', var_name='Year')
df_scope2_loc = df_scope2_loc.melt(id_vars='Company Name', value_name='Scope_2_emissions_(tCO₂e)_Location', var_name='Year')
df_scope2_mkt = df_scope2_mkt.melt(id_vars='Company Name', value_name='Scope_2_emissions_(tCO₂e)_Market', var_name='Year')

In [193]:
companies_sample = pd.read_csv("CompaniesSample.csv")

df = df.merge(companies_sample['Company Name'], on='Company Name', how='inner')
df.replace('', '-', inplace=True)
df.to_sql('Emissions_Data_Joined', engine, if_exists='replace', index=False)

400

In [121]:
df = pd.read_sql_table('Emissions_Data_Joined', engine)

In [122]:
df[['Year', 'Scope_1_emissions_(tCO₂e)']].loc[df['Scope_1_emissions_(tCO₂e)']!='-'].astype(int).groupby('Year').describe()

Scope_1_emissions_(tCO₂e)                                               \
                         count          mean           std    min       25%   
Year                                                                          
2018                      75.0  2.348292e+06  8.122664e+06  306.0  14189.00   
2019                      83.0  2.037170e+06  6.738080e+06  129.0  17119.50   
2020                      88.0  2.000004e+06  6.274668e+06  169.0   8894.75   
2021                      91.0  1.757240e+06  5.369972e+06  152.0  15536.50   

                                       
           50%        75%         max  
Year                                   
2018  132360.0  375698.00  57205670.0  
2019   99193.0  366099.50  46188978.0  
2020   81261.0  321261.75  38589016.0  
2021   82846.0  362425.50  37528756.0

In [123]:
df[['Year', 'Scope_2_emissions_(tCO₂e)_Location']].loc[df['Scope_2_emissions_(tCO₂e)_Location']!='-'].astype(int).groupby('Year').describe()

Scope_2_emissions_(tCO₂e)_Location                                       \
                                  count           mean           std     min   
Year                                                                           
2018                               74.0  848826.216216  2.053190e+06  2347.0   
2019                               80.0  732786.662500  1.691818e+06  2407.0   
2020                               84.0  674666.869048  1.522946e+06   116.0   
2021                               87.0  698196.022989  1.482567e+06  1794.0   

                                                 
           25%       50%        75%         max  
Year                                             
2018  39120.75  193431.5  595960.25  13057352.0  
2019  36504.75  203867.0  571340.75  12190878.0  
2020  26505.25  178575.5  495295.75   9891120.0  
2021  26348.50  159638.0  495345.50   9475167.0

In [124]:
df[['Year', 'Scope_2_emissions_(tCO₂e)_Market']].loc[df['Scope_2_emissions_(tCO₂e)_Market']!='-'].astype(int).groupby('Year').describe()

Scope_2_emissions_(tCO₂e)_Market                                      \
                                count           mean           std    min   
Year                                                                        
2018                             54.0  672488.425926  1.722814e+06  163.0   
2019                             60.0  579992.133333  1.538167e+06   10.0   
2020                             63.0  491769.904762  1.197243e+06    1.0   
2021                             67.0  470143.582090  1.006411e+06  237.0   

                                                 
           25%       50%        75%         max  
Year                                             
2018  18703.25  175106.5  633356.25  12022083.0  
2019  19737.75  136197.0  473529.00  11078980.0  
2020   8069.50   42766.0  402806.50   8108016.0  
2021  11638.00  113019.0  393352.00   6544173.0